In [1]:
import pandas as pd
import numpy as np


In [2]:
col_names = ["buying", "maint", "doors", "persons", "lug_boot", "safety", "label"]

train_data = pd.read_csv('train.csv',names=col_names)
test_data = pd.read_csv('test.csv',names=col_names)

train_data.head()

,buying,maint,doors,persons,lug_boot,safety,label
0,low,vhigh,4,4,big,med,acc
1,low,high,5more,4,med,high,vgood
2,vhigh,med,2,2,big,high,unacc
3,high,high,2,2,small,high,unacc
4,vhigh,low,3,2,big,low,unacc


In [3]:
# all different info gain helper methods
def entropy(target_attribute):
    elements, count = np.unique(target_attribute, return_counts=True)
    entropy = -np.sum([(count[i]/np.sum(count))*np.log2(count[i]/np.sum(count)) for i in range (len(elements))])
    return entropy

In [4]:
# now helper for Majority Error
def majority_error(target_attribute):
    _, count = np.unique(target_attribute, return_counts=True)
    majority = np.max(count)
    return 1 - majority/np.sum(count)

In [5]:
# helper for gini index
def gini_index(target_attribute):
    elements, count = np.unique(target_attribute, return_counts=True)
    giniboi = 1 - np.sum([(count[i]/np.sum(count))**2 for i in range(len(elements))])
    return giniboi

In [6]:
# now calculating the info gain
def infomation_gain(data, split_attribute_name, target_name="label" , citerion="entropy"):
    if citerion == "entropy":
        total_entropy = entropy(data[target_name])
    elif citerion == "majority_error":
        total_entropy = majority_error(data[target_name])
    elif citerion == "gini_index":
        total_entropy = gini_index(data[target_name])
    else:
        raise ValueError("invalid citerion, check spelling or put valid entry")
    
    vals, counts = np.unique(data[split_attribute_name], return_counts=True)
    weighted_entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_attribute_name]==vals[i]).dropna()[target_name]) for i in range(len(vals))])
    
    return total_entropy - weighted_entropy
        

In [7]:
# ID3 Algorithm, 

def ID3(data, og_data, features, target_attribute_name = "label", parent_node_class = None, max_depth = None, depth = 0, criterion = "entropy"):
    # base cases
  
    #if all the labels in the subseted data are the same return that label (leaf node)
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]
    
    # now the base case when the data is empty return the max label
    elif (len(data) == 0):
        return np.unique(og_data[target_attribute_name])[np.argmax(np.unique(og_data[target_attribute_name], return_counts=True)[1])]
    
    #now there is a third base case for the generalizer. It is the parent_node_class that assigns a label if a leaf node label cannot be decided
    #This happens if attributes (features) run out and also when a max depth is reached 
    elif len(features) == 0 or (max_depth and depth == max_depth):
        return parent_node_class

    else:
        # create root node for tree
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])]
    
       #find infomation gains for each colomn
        info_values = [infomation_gain(data, feature, target_attribute_name, criterion) for feature in features]
        #find the highest gain
        best_feature_index = np.argmax(info_values)
        next_split = features[best_feature_index]
        
        #take away the feature used to split so that info gain is not calculated on it
        features = [i for i in features if i != next_split]

        #dictionary for tree to build and return
        tree = {next_split:{}}
        
        #itterate through values in next split and begin the recursion
        for val in np.unique(data[next_split]):
            subset_data = data.where(data[next_split] == val).dropna()
            sub_tree = ID3(subset_data, og_data, features, target_attribute_name, parent_node_class, max_depth, depth+1, criterion)
            tree[next_split][val] = sub_tree

    return tree




In [8]:

def predict(toPredict, tree, default=1):
    # Iterate over each key in the query
    for key in toPredict.keys():
       
        if key in tree.keys():
            try:
                #use the decision tree to predict value
                result = tree[key][toPredict[key]]
            except:
                
                return default

            # now go in the tree recuseively to go down sub-branches to find a leaf node
            if isinstance(result, dict):
                return predict(toPredict, result)
        
            else:
                return result
            

In [9]:
#This method sees how accurate the tree is with the data 
def test(data, tree):
    
    #makes the dataframe into a dictionary
    #use orient = records to use all columns except last one
    data_dic = data.iloc[:, :-1].to_dict(orient="records")

    #making a df to store predicted values
    predicted_vals = pd.DataFrame(columns=["predicted"])

    #now fill the predicted df for each value of data
    for i in range(len(data)):
        predict_value = predict(data_dic[i], tree, 1.0)
        predicted_vals.loc[i, "predicted"] = predict_value

    # now make an accuracy percentage of the percentages that were right
    correct_predictions = np.sum(predicted_vals["predicted"] == data["label"])
    accuracy_score = (correct_predictions / len(data)) * 100

    return accuracy_score

In [10]:
# this funtion computes the error for the test and training data and uses the tree constructed in the ID3 algorithm
def compute_error(tree, train_data, test_data):
   
    train_acc = test(train_data, tree)
    test_acc = test(test_data, tree)

    # since test returns the percentage values that were correct do 100 - right to get the error
    train_error = 100 - train_acc
    test_error = 100 - test_acc

    return train_error, test_error

In [11]:
# Now use all methods to run the experiment to find the test error for each critereion
criterions = ["entropy", "majority_error", "gini_index"]
max_depths = [1,2,3,4,5,6]
results = []


for criterion in criterions:
    for max_depth in max_depths:
        tree = ID3(train_data, train_data, train_data.columns[:-1], max_depth=max_depth, criterion=criterion)
        train_error, test_error = compute_error(tree, train_data, test_data)
        results.append((criterion, max_depth, train_error, test_error))

#results of the TESTS
for criterion, depth, train_err, test_err in results:
    print(f"Criterion_Car: {criterion}, Depth_Car: {depth}, Training Error Car: {train_err:.6f}, Test Error Car: {test_err:.6f}")

Criterion_Car: entropy, Depth_Car: 1, Training Error Car: 30.200000, Test Error Car: 29.670330
Criterion_Car: entropy, Depth_Car: 2, Training Error Car: 30.200000, Test Error Car: 29.670330
Criterion_Car: entropy, Depth_Car: 3, Training Error Car: 22.200000, Test Error Car: 22.252747
Criterion_Car: entropy, Depth_Car: 4, Training Error Car: 13.000000, Test Error Car: 14.835165
Criterion_Car: entropy, Depth_Car: 5, Training Error Car: 3.300000, Test Error Car: 11.401099
Criterion_Car: entropy, Depth_Car: 6, Training Error Car: 0.000000, Test Error Car: 12.500000
Criterion_Car: majority_error, Depth_Car: 1, Training Error Car: 30.200000, Test Error Car: 29.670330
Criterion_Car: majority_error, Depth_Car: 2, Training Error Car: 30.200000, Test Error Car: 29.670330
Criterion_Car: majority_error, Depth_Car: 3, Training Error Car: 22.200000, Test Error Car: 22.252747
Criterion_Car: majority_error, Depth_Car: 4, Training Error Car: 13.000000, Test Error Car: 14.835165
Criterion_Car: majority_

In [12]:
column_names = ["age", "job", "marital", "education", "default", "balance", "housing", "loan", "contact", "day_of_week", "month", "duration", "campaign", "pdays", "previous", "poutcome", "y"]

bank_train = pd.read_csv('bank/train.csv', names=column_names)
bank_test = pd.read_csv('bank/test.csv', names=column_names)
bank_train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day_of_week,month,duration,campaign,pdays,previous,poutcome,y
0,41,services,married,secondary,no,0,yes,no,unknown,5,may,114,2,-1,0,unknown,no
1,48,blue-collar,single,secondary,no,312,yes,yes,cellular,3,feb,369,2,-1,0,unknown,no
2,55,technician,married,secondary,no,1938,no,yes,cellular,18,aug,193,1,386,3,success,yes
3,54,admin.,married,tertiary,no,59,yes,no,cellular,10,jul,268,1,-1,0,unknown,no
4,34,management,single,tertiary,no,2646,no,no,cellular,14,apr,142,1,-1,0,unknown,yes


In [13]:


# convert numberical value to a binary
# we find the medium of the column and then that is the threshold 
# cols have numerical: age, balance, day_of_week, duration, campain, pdays, previous
bank_train_data_fixed = bank_train
bank_test_data_fixed = bank_test
NumCols=['age', 'balance', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous']
def changeNumbericToBinary(ListcolName):
    bank_train_data_fixed = bank_train.copy()
    bank_test_data_fixed = bank_test.copy()

    # Find median for each column in the list
    for colName in ListcolName:
        threshold_train = bank_train_data_fixed[colName].median()
        threshold_test = bank_test_data_fixed[colName].median()


    # Map the values to 'less' or 'bigger' based on median
    bank_train_data_fixed[colName] = bank_train_data_fixed[colName].apply(lambda x: 'less' if x < threshold_train else 'bigger')
    bank_test_data_fixed[colName] = bank_test_data_fixed[colName].apply(lambda x: 'less' if x < threshold_test else 'bigger')

    return bank_train_data_fixed, bank_test_data_fixed

In [14]:
NumCols=['age', 'balance', 'day_of_week', 'duration', 'campaign', 'pdays', 'previous']
bank_train_data_fixed, bank_test_data_fixed = changeNumbericToBinary(NumCols)

bank_test_data_fixed.head()

,age,job,marital,education,default,balance,housing,loan,contact,day_of_week,month,duration,campaign,pdays,previous,poutcome,y
0,41,management,single,secondary,no,764,no,no,cellular,12,jun,230,2,-1,bigger,unknown,no
1,39,blue-collar,married,secondary,no,49,yes,no,cellular,14,may,566,1,370,bigger,failure,no
2,60,retired,married,primary,no,0,no,no,telephone,30,jul,130,3,-1,bigger,unknown,no
3,31,entrepreneur,single,tertiary,no,247,yes,yes,unknown,2,jun,273,1,-1,bigger,unknown,no
4,26,student,single,unknown,no,2020,no,no,telephone,28,jan,42,3,-1,bigger,unknown,no


In [15]:
# ID3 Algorithm, fix the target attribute so that it is y

def ID3(data, og_data, features, target_attribute_name = "y", parent_node_class = None, max_depth = None, depth = 0, criterion = "entropy"):
    # base cases

    #if all the labels in the subseted data are the same return that label (leaf node)
    if len(np.unique(data[target_attribute_name])) <= 1:
        return np.unique(data[target_attribute_name])[0]

    # now the base case when the data is empty return the max label
    elif (len(data) == 0):
        return np.unique(og_data[target_attribute_name])[np.argmax(np.unique(og_data[target_attribute_name], return_counts=True)[1])]

    #now there is a third base case for the generalizer. It is the parent_node_class that assigns a label if a leaf node label cannot be decided
    #This happens if attributes (features) run out and also when a max depth is reached 
    elif len(features) == 0 or (max_depth and depth == max_depth):
        return parent_node_class

    else:
        # create root node for tree
        parent_node_class = np.unique(data[target_attribute_name])[np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])]

        #find infomation gains for each colomn
        info_values = [infomation_gain(data, feature, target_attribute_name, criterion) for feature in features]
        #find the highest gain
        best_feature_index = np.argmax(info_values)
        next_split = features[best_feature_index]

        #take away the feature used to split so that info gain is not calculated on it
        features = [i for i in features if i != next_split]

        #dictionary for tree to build and return
        tree = {next_split:{}}

        #itterate through values in next split and begin the recursion
        for val in np.unique(data[next_split]):
            subset_data = data.where(data[next_split] == val).dropna()
            sub_tree = ID3(subset_data, og_data, features, target_attribute_name, parent_node_class, max_depth, depth+1, criterion)
            tree[next_split][val] = sub_tree

    return tree


In [16]:
#This method sees how accurate the tree is with the data 
def test(data, tree):

    #makes the dataframe into a dictionary
    #use orient = records to use all columns except last one
    data_dic = data.iloc[:, :-1].to_dict(orient="records")

    #making a df to store predicted values
    predicted_vals = pd.DataFrame(columns=["predicted"])

    #now fill the predicted df for each value of data
    for i in range(len(data)):
        predict_value = predict(data_dic[i], tree, 1.0)
        predicted_vals.loc[i, "predicted"] = predict_value

    # now make an accuracy percentage of the percentages that were right
    correct_predictions = np.sum(predicted_vals["predicted"] == data["y"])
    accuracy_score = (correct_predictions / len(data)) * 100

    return accuracy_score

In [ ]:
criterions = ["entropy", "majority_error", "gini_index"]
banks_max_depth = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
results = []


for criterion in criterions:
    for max_depth in banks_max_depth:
        tree = ID3(bank_train_data_fixed, bank_train_data_fixed, bank_train_data_fixed.columns[:-1], max_depth=max_depth, criterion=criterion)
        train_error, test_error = compute_error(tree, bank_train_data_fixed, bank_test_data_fixed)
        results.append((criterion, max_depth, train_error, test_error))

#results of the TESTS
for criterion, depth, train_err, test_err in results:
    print(f"Criterion_bank: {criterion}, Depth_bank: {depth}, Training Error bank: {train_err:.6f}, Test Error bank: {test_err:.6f}")

In [ ]:
# find columns with unknown as a value
columns_with_unknown = bank_train_data_fixed.apply(lambda col: col.apply(lambda x: x == 'unknown')).any()

unknowns = columns_with_unknown[columns_with_unknown].index.tolist()
print(unknowns)

In [ ]:
bank_train_data_fixed_noNA = bank_train_data_fixed.copy()
bank_test_data_fixed_noNA = bank_test_data_fixed.copy()
cols_with_unknown = ['job', 'education', 'contact', 'poutcome']
for unknowns in cols_with_unknown:
    #     mod_value = bank_test_data_fixed[bank_train_data_fixed[unknowns] != 'unknown'][unknowns].mode()[0]
    #     bank_train_data_fixed[unknowns] = bank_train_data_fixed[unknowns].apply(lambda x: mod_value if x = 'unknown')
    # Check if the column has 'unknown' values

    # Calculate the mode (most frequent value) excluding 'unknown'
    mode_val = bank_train_data_fixed_noNA[unknowns][bank_train_data_fixed_noNA[unknowns] != 'unknown'].mode().values[0]
    mode_val2 = bank_test_data_fixed_noNA[unknowns][bank_test_data_fixed_noNA[unknowns] != 'unknown'].mode().values[0]
    # Replace 'unknown' values with the mode
    bank_train_data_fixed_noNA[unknowns] = bank_train_data_fixed_noNA[unknowns].replace('unknown', mode_val)
    bank_test_data_fixed_noNA[unknowns] = bank_test_data_fixed_noNA[unknowns].replace('unknown', mode_val2)

In [ ]:
#check to see if unknowns were replaced
columns_with_unknown2 = bank_train_data_fixed_noNA.columns[bank_train_data_fixed_noNA.apply(lambda col: col.str.contains('unknown')).any()]
print(columns_with_unknown2)

In [ ]:
criterions = ["entropy", "majority_error", "gini_index"]
banks_max_depth = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
results = []


for criterion in criterions:
    for max_depth in banks_max_depth:
        tree = ID3(bank_train_data_fixed_noNA, bank_train_data_fixed_noNA, bank_train_data_fixed_noNA.columns[:-1], max_depth=max_depth, criterion=criterion)
        train_error, test_error = compute_error(tree, bank_train_data_fixed_noNA, bank_test_data_fixed_noNA)
        results.append((criterion, max_depth, train_error, test_error))

#results of the TESTS
for criterion, depth, train_err, test_err in results:
    print(f"Criterion_bank: {criterion}, Depth_bank: {depth}, Training Error bank: {train_err:.6f}, Test Error bank: {test_err:.6f}")